In [1]:
# Librerías.
import pandas as pd
import Funciones as f
import unidecode
from docx import Document
from docx.shared import Inches
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats

In [2]:
%%capture
%run "40. Gráficos de barras de tiempos.ipynb"

In [3]:
def Crear_Tabla_P_Valores(Diccionario_P_Valores, Nombre_Archivo='tabla_p_valores.png'):
    
    """
    Crea una tabla formateada con p-valores coloreados según significancia estadística.
    Los valores menores a 0.05 se muestran en verde, los mayores en rojo.
    Todos los valores se formatean en notación científica.
    
    """
    
    # Extraer todos los números de ítems de todas las claves.
    Numeros_Items = set()
    
    for Categoria in Diccionario_P_Valores.values():
        for Clave in Categoria.keys():
            # Buscar patrones como 'CO_Item_X_Der', 'CT_Item_X_Izq', etc.
            if ('CO_Item_' in Clave or 'CT_Item_' in Clave):
                # Extraer número del ítem
                Partes = Clave.split('_')
                if len(Partes) >= 3 and Partes[2].isdigit():
                    Numeros_Items.add(int(Partes[2]))
    
    Numeros_Items = sorted(list(Numeros_Items))
    
    # Crear DataFrame base.
    Datos_Tabla = []
    
    for Numero_Item in Numeros_Items:
        Fila = {'Item': Numero_Item}
        
        # Para cada categoría, buscar las 4 combinaciones: Opinion Izq/Der y Tiempo Izq/Der
        for Categoria in Diccionario_P_Valores.keys():
            # Claves de Opinión
            Clave_Opinion_Izq = f'CO_Item_{Numero_Item}_Izq'
            Clave_Opinion_Der = f'CO_Item_{Numero_Item}_Der'
            
            # Claves de Tiempo
            Clave_Tiempo_Izq = f'CT_Item_{Numero_Item}_Izq'
            Clave_Tiempo_Der = f'CT_Item_{Numero_Item}_Der'
            
            # Agregar las 4 columnas por categoría
            Fila[f'Opinion Izq ({Categoria})'] = Diccionario_P_Valores[Categoria].get(Clave_Opinion_Izq, np.nan)
            Fila[f'Opinion Der ({Categoria})'] = Diccionario_P_Valores[Categoria].get(Clave_Opinion_Der, np.nan)
            Fila[f'Tiempo Izq ({Categoria})'] = Diccionario_P_Valores[Categoria].get(Clave_Tiempo_Izq, np.nan)
            Fila[f'Tiempo Der ({Categoria})'] = Diccionario_P_Valores[Categoria].get(Clave_Tiempo_Der, np.nan)
        
        Datos_Tabla.append(Fila)
    
    Df_Tabla = pd.DataFrame(Datos_Tabla)
    
    # Crear la figura con más ancho para acomodar las 8 columnas
    Fig, Ax = plt.subplots(figsize=(20, len(Numeros_Items) * 0.4 + 3))
    Ax.axis('tight')
    Ax.axis('off')
    
    # Preparar datos para la tabla con formateo
    Datos_Formateados = []
    Colores_Celdas = []
    
    for _, Fila in Df_Tabla.iterrows():
        Fila_Formateada = [str(int(Fila['Item']))]
        Colores_Fila = ['white']  # Color para columna Item
        
        # Formatear columnas de p-valores
        Columnas_P_Valores = [col for col in Df_Tabla.columns if col != 'Item']
        
        for Columna in Columnas_P_Valores:
            Valor = Fila[Columna]
            if pd.isna(Valor):
                Fila_Formateada.append('N/A')
                Colores_Fila.append('lightgray')
            else:
                # Formatear en notación científica con 3 decimales
                Valor_Formateado = f'{Valor:.3e}'
                Fila_Formateada.append(Valor_Formateado)
                
                # Determinar color según significancia
                if Valor < 0.05:
                    Colores_Fila.append('lightgreen')
                else:
                    Colores_Fila.append('lightcoral')
        
        Datos_Formateados.append(Fila_Formateada)
        Colores_Celdas.append(Colores_Fila)
    
    # Crear encabezados
    Encabezados = ['Item'] + [col for col in Df_Tabla.columns if col != 'Item']
    
    # Crear la tabla
    Tabla = Ax.table(cellText=Datos_Formateados,
                     colLabels=Encabezados,
                     cellColours=Colores_Celdas,
                     cellLoc='center',
                     loc='center')
    
    # Configurar estilo
    Tabla.auto_set_font_size(False)
    Tabla.set_fontsize(6)
    Tabla.scale(1.0, 1.3)
    
    # Estilizar encabezados
    for i in range(len(Encabezados)):
        Tabla[(0, i)].set_facecolor('#4CAF50')
        Tabla[(0, i)].set_text_props(weight='bold', color='white')
    
    plt.title('Tabla de P-Valores por Ítem y Categoría\n' + 
              '(Verde: p < 0.05, Rojo: p ≥ 0.05)', 
              fontsize=14, fontweight='bold', pad=20)
    
    plt.tight_layout()
    plt.savefig(Nombre_Archivo, dpi=300, bbox_inches='tight')
    plt.show()
    
    return Df_Tabla

In [ ]:
Tabla_Resultados = Crear_Tabla_P_Valores(Diccionario_P_Valores)